## Run model inference interactively

In [1]:
%%bash 
pwd
echo $MODELS_DIR

/home/a2si2021/tf_object_detection/notebooks
/home/a2si2021/tf_object_detection/workspace/cloud_detection_20210915/models


In [5]:
import os
import glob
from pathlib import Path

model_name = "faster_rcnn_resnet152_v1_640x640_coco17_tpu-8_cloud_detection_dataset_5-class_2021_09_15_2021_09_15"

PATH_TO_SAVED_MODEL = Path(os.environ['EXPORT_DIR']) / Path(model_name + '_ckpt-31') / Path('saved_model')
PATH_TO_LABEL_MAP = Path(os.environ['ANNOT_DIR']) / Path(model_name) / Path('label_map.pbtxt')
PATH_TO_TEST_IMAGES_DIR = Path(os.environ['NOTEBOOK_DIR']) / Path('input')
PATH_TO_OUTPUT_IMAGES_DIR = Path(os.environ['NOTEBOOK_DIR']) / Path('output')

assert os.path.isdir(PATH_TO_SAVED_MODEL)
assert os.path.isfile(PATH_TO_LABEL_MAP)
assert os.path.isdir(PATH_TO_TEST_IMAGES_DIR)
assert os.path.isdir(PATH_TO_OUTPUT_IMAGES_DIR)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.jpg"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)

num_images = len(TEST_IMAGE_PATHS)
print(num_images)

138


In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import tensorflow as tf
tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

print('BEGIN loading model')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(str(PATH_TO_SAVED_MODEL))

end_time = time.time()
elapsed_time = end_time - start_time
print('DONE loading model. Took {:.2f} seconds'.format(elapsed_time))

BEGIN loading model
DONE loading model. Took 28.84 seconds


In [7]:




label_map = label_map_util.load_labelmap(str(PATH_TO_LABEL_MAP))
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=len(label_map.item), use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


def run_inference_for_single_image(image_path, detect_fn):
    print('BEGIN running inference for {}'.format(image_path))
    image_np = load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {
        key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        #instance_masks=detections.get('detection_masks'),
        use_normalized_coordinates=True,
        max_boxes_to_draw=100,
        #line_thickness=8,
        min_score_thresh=0.50,
        agnostic_mode=False)

    # below is for debug
    im_height, im_width, im_num_ch = image_np.shape
    #for idx, box in enumerate(detections['detection_boxes']):
    #    if detections['detection_scores'][idx] >= 0.5:
    #        [ymin, xmin, ymax, xmax] = box
    #        (left, right, top, bottom) = (
    #            int(xmin * im_width), int(xmax * im_width),
    #            int(ymin * im_height), int(ymax * im_height))
    #        print("(left, right, top, bottom) = ({}, {}, {}, {})".format(left, right, top, bottom))
    #plt.figure()
    #plt.imshow(image_np_with_detections)
    im = Image.fromarray(image_np_with_detections)
    im.save(Path(PATH_TO_OUTPUT_IMAGES_DIR) / Path(os.path.basename(image_path)))
    print('DONE running inference for {}'.format(image_path))

    return detections

start_time = time.time()
print('BEGIN running inference for testing dataset')
for image_idx, image_path in enumerate(TEST_IMAGE_PATHS):
    print('PROGRESS: {}/{}'.format(image_idx+1, num_images))
    output_dict = run_inference_for_single_image(image_path, detect_fn)

end_time = time.time()
elapsed_time = end_time - start_time
elapsed_time_per_image = elapsed_time/num_images
print('DONE running inference for testing dataset. Took {:.2f} seconds per image'.format(elapsed_time_per_image))

BEGIN running inference for testing dataset
PROGRESS: 1/138
BEGIN running inference for /home/a2si2021/tf_object_detection/notebooks/input/R2W_D27502612_20210722130234542_TIMING.jpg
DONE running inference for /home/a2si2021/tf_object_detection/notebooks/input/R2W_D27502612_20210722130234542_TIMING.jpg
PROGRESS: 2/138
BEGIN running inference for /home/a2si2021/tf_object_detection/notebooks/input/R1E_D27502592_20210831115252814_TIMING.jpg
DONE running inference for /home/a2si2021/tf_object_detection/notebooks/input/R1E_D27502592_20210831115252814_TIMING.jpg
PROGRESS: 3/138
BEGIN running inference for /home/a2si2021/tf_object_detection/notebooks/input/R2E_D27502528_20210828162257239_TIMING.jpg
DONE running inference for /home/a2si2021/tf_object_detection/notebooks/input/R2E_D27502528_20210828162257239_TIMING.jpg
PROGRESS: 4/138
BEGIN running inference for /home/a2si2021/tf_object_detection/notebooks/input/T3SW_D27502473_20210907120254146_TIMING.jpg
DONE running inference for /home/a2si202